In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import statistics as st

In [2]:
# Load data and normalize pixel values
data = sio.loadmat('mnist_10digits.mat')
xtrain = data['xtrain'].T/255
ytrain = data['ytrain']

In [3]:
def kmeans_l2(data):
    # data: 784 x m array of handwritten digits
    m = data.shape[1]
    k = 10
    # Randomly select k data points to use as initial centroids
    centroid = xtrain[:, np.random.choice(m, k)]
    
    c_old = np.copy(centroid) + 10
    n_iter = 0 # Track number of iterations
    
    while np.linalg.norm(centroid - c_old, ord='fro') > 1e-2:
        # Assign each data point to a centroid
        c2 = np.sum(np.power(centroid,2), axis=0, keepdims=True)
        diff = -2*np.dot(data.T, centroid) + c2
        labels = np.argmin(diff, axis=1)
        
        # Adjust the centroid locations
        c_old = np.copy(centroid)
        bEmpty = False # Boolean for tracking whether any clusters are empty
        
        for c in range(centroid.shape[1]):
            count = np.count_nonzero(labels == c)
            if count == 0:
                bEmpty = True
                continue
            avg = np.mean(data.T[labels == c], axis=0)
            centroid[:,c] = avg
            
        if bEmpty:
            # Find the empty clusters
            c_empty = np.delete(np.arange(0,k,1),np.unique(labels))
            # Remove the centroids and recalculate k
            centroid = np.delete(centroid, c_empty, axis=0)
            c_old = np.delete(c_old, c_empty, axis=0)
            k = centroid.shape[1]
        
        n_iter += 1
        
    # Calculate purity of each cluster
    purity = []
    for c in np.unique(labels):
        # Get actual labels for points in each cluster from ytrain
        true_labels = ytrain.reshape(60000)[labels == c]
        mode = st.mode(true_labels)
        p = len(true_labels[true_labels == mode])/len(true_labels)
        purity.append(p)
    
    return labels, centroid, n_iter, purity

In [4]:
labels, centroid, n_iter, purity = kmeans_l2(xtrain)

In [5]:
for p in range(len(purity)):
    print('Purity of cluster {}: {}'.format(p,round(purity[p],3)))

Purity of cluster 0: 0.508
Purity of cluster 1: 0.516
Purity of cluster 2: 0.424
Purity of cluster 3: 0.525
Purity of cluster 4: 0.562
Purity of cluster 5: 0.755
Purity of cluster 6: 0.351
Purity of cluster 7: 0.919
Purity of cluster 8: 0.663
Purity of cluster 9: 0.404


In [6]:
def kmeans_l1(data):
    # K-means clustering using Manhattan distance
    # data: 784 x m array of handwritten digits
    m = data.shape[1]
    k = 10
    # Randomly select k data points to use as initial centroids
    centroid = xtrain[:, np.random.choice(m, k)]
    
    c_old = np.copy(centroid) + 10
    n_iter = 0 # Track number of iterations
    
    while np.linalg.norm(centroid - c_old, ord='fro') > 1e-1:
        diff = np.ndarray((centroid.shape[1],m), dtype=np.double)
        # Assign each data point to a centroid
        for c in range(centroid.shape[1]):
            diff[c,:] = np.sum(np.absolute(data - centroid[:,c].reshape(784,1)), axis=0)
        labels = np.argmin(diff, axis=0)
        
        # Adjust the centroid locations
        c_old = np.copy(centroid)
        bEmpty = False # Boolean for tracking whether any clusters are empty
        
        for c in range(centroid.shape[1]):
            count = np.count_nonzero(labels == c)
            if count == 0:
                bEmpty = True
                continue
            #totals = np.sum(data.T[labels == c], axis=0)
            #avg = totals/count
            avg = np.median(data.T[labels == c], axis=0)
            centroid[:,c] = avg
            
        if bEmpty:
            # Find the empty clusters
            c_empty = np.delete(np.arange(0,k,1),np.unique(labels))
            # Remove the centroids and recalculate k
            centroid = np.delete(centroid, c_empty, axis=0)
            c_old = np.delete(c_old, c_empty, axis=0)
            k = centroid.shape[1]
        
        n_iter += 1
        
    # Calculate purity of each cluster
    purity = []
    for c in np.unique(labels):
        # Get actual labels for points in each cluster from ytrain
        true_labels = ytrain.reshape(60000)[labels == c]
        mode = st.mode(true_labels)
        p = len(true_labels[true_labels == mode])/len(true_labels)
        purity.append(p)
        
    return labels, centroid, n_iter, purity

In [7]:
# Note: may take several minutes to run
labels, centroid, n_iter, purity = kmeans_l1(xtrain)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [8]:
for p in range(len(purity)):
    print('Purity of cluster {}: {}'.format(p,round(purity[p],3)))

Purity of cluster 0: 0.747
Purity of cluster 1: 0.41
Purity of cluster 2: 0.214
Purity of cluster 3: 0.583
Purity of cluster 4: 0.503
Purity of cluster 5: 0.533
Purity of cluster 6: 0.494
Purity of cluster 7: 0.929
Purity of cluster 8: 0.558
Purity of cluster 9: 0.376
